# Pandas

In [4]:
import pandas as pd
import numpy as np
print("Pandas version: {}".format(pd.__version__))

Pandas version: 1.4.3


In [6]:
# a sample series
s = pd.Series(list("abcdef"), index=[49, 48, 47, 0, 1, 2])
print(s)
# a sample dataframe
df = pd.DataFrame(np.arange(25).reshape(5, 5),  
                      index=list('abcde'), 
                      columns=['x','y','z', 8, 9])
print(df)

49    a
48    b
47    c
0     d
1     e
2     f
dtype: object
    x   y   z   8   9
a   0   1   2   3   4
b   5   6   7   8   9
c  10  11  12  13  14
d  15  16  17  18  19
e  20  21  22  23  24


## Slicing using loc and iloc

The main distinction between the two methods is:

1. loc gets rows (and/or columns) with particular labels/index/conditions.

2. iloc gets rows (and/or columns) at integer locations (only integer).

In [8]:
assert s.loc[0] == 'd'
assert s.iloc[0] == 'a'
# rows at index labels between 0 and 1 (inclusive)
print(s.loc[0:1])
# rows at index location betwen 0 and 1 (left inclusive, right exclusive)
print(s.iloc[0:1])

0    d
1    e
dtype: object
49    a
dtype: object


In [32]:
s.loc[s.index[3]]

'd'

In [30]:
s.iloc[s.index[3]]

'a'

Here are some of the differences/similarities between s.loc and s.iloc when passed various objects:

| <object>               | description                                                  | `s.loc[<object>]`                             | `s.iloc[<object>]`                             |
| :--------------------- | :----------------------------------------------------------- | :-------------------------------------------- | :--------------------------------------------- |
| `0`                    | single item                                                  | Value at index *label* `0` (the string `'d'`) | Value at index *location* 0 (the string `'a'`) |
| `0:1`                  | slice                                                        | **Two** rows (labels `0` and `1`)             | **One** row (first row at location 0)          |
| `1:47`                 | slice with out-of-bounds end                                 | **Zero** rows (empty Series)                  | **Five** rows (location 1 onwards)             |
| `1:47:-1`              | slice with negative step                                     | **three** rows (labels `1` back to `47`)      | **Zero** rows (empty Series)                   |
| `[2, 0]`               | integer list                                                 | **Two** rows with given labels                | **Two** rows with given locations              |
| `s > 'e'`              | Bool series (indicating which values have the property)      | **One** row (containing `'f'`)                | `NotImplementedError`                          |
| `(s>'e').values`       | Bool array                                                   | **One** row (containing `'f'`)                | Same as `loc`                                  |
| `999`                  | int object not in index                                      | `KeyError`                                    | `IndexError` (out of bounds)                   |
| `-1`                   | int object not in index                                      | `KeyError`                                    | Returns last value in `s`                      |
| `lambda x: x.index[3]` | callable applied to series (here returning 3rd item in index) | `s.loc[s.index[3]]`                           | `s.iloc[s.index[3]]`                           |

| <object>               | description                                                  | `s.loc[<object>]` | `s.iloc[<object>]` |
| :--------------------- | :----------------------------------------------------------- | ----------------- | ------------------ |
| `0`                    | single item                                                  | d                 | a                  |
| `0:1`                  | slice                                                        | de                | a                  |
| `1:47`                 | slice with out-of-bounds end                                 | EMPTY             | bcdef              |
| `1:47:-1`              | slice with negative step                                     | edc               | Error              |
| `[2, 0]`               | integer list                                                 | fd                | ca                 |
| `s > 'e'`              | Bool series (indicating which values have the property)      | f                 | Error              |
| `(s>'e').values`       | Bool array                                                   | f                 | f                  |
| `999`                  | int object not in index                                      | EMPTY             | EMPTY              |
| `-1`                   | int object not in index                                      | Error             | f                  |
| `lambda x: x.index[3]` | callable applied to series (here returning 3rd item in index) | d                 | a                  |

Using **loc[]** to get a slice between two labels. Note that the labels are not necessarily in a natural order.

In [36]:
s.loc[48:2] # both range inclusive

48    b
47    c
0     d
1     e
2     f
dtype: object

loc[ ] works well with DateTime indexes. We don't need to pass the exact date/time to fetch by label.

In [37]:
s3 = pd.Series(list('abcde'), pd.date_range('now', periods=5, freq='M')) 
s3

2022-10-31 17:06:59.108838    a
2022-11-30 17:06:59.108838    b
2022-12-31 17:06:59.108838    c
2023-01-31 17:06:59.108838    d
2023-02-28 17:06:59.108838    e
Freq: M, dtype: object

In [40]:
# To get the rows between two months
s3.loc['2022-03':'2022-12']

2022-10-31 17:06:59.108838    a
2022-11-30 17:06:59.108838    b
2022-12-31 17:06:59.108838    c
Freq: M, dtype: object

## Working with DataFrame

loc and iloc work the same way with DataFrames as they do with Series. It's useful to note that both methods can address columns and rows together.

When given a tuple, the first element is used to index the rows and, if it exists, the second element is used to index the columns.

To get a subset of DF: from rows 'c' and onwards AND columns up to 'z'

In [41]:
df.loc['c': , :'z']  # rows 'c' and onwards AND columns up to 'z'

,x,y,z
c,10,11,12
d,15,16,17
e,20,21,22


To get a subset: all rows and only the third column

In [45]:
df.iloc[:, 3]

a     3
b     8
c    13
d    18
e    23
Name: 8, dtype: int64

To get a subset: all rows and only the second and third column

In [43]:
df.iloc[:, 2:4]

,z,8
a,2,3
b,7,8
c,12,13
d,17,18
e,22,23


**Mixing labels and numbers in slicing**: how best to slice the rows up to and including 'c' and take the first four columns?

In [47]:
# method 1
df.loc[:'c', df.columns[0:4]]

,x,y,z,8
a,0,1,2,3
b,5,6,7,8
c,10,11,12,13


In [48]:
# method 2
df.iloc[:df.index.get_loc('c') + 1, :4]

,x,y,z,8
a,0,1,2,3
b,5,6,7,8
c,10,11,12,13


## Advanced slicing

Keeping the rows with 'x' column greater than 5 and columns of 'z' and '8'?

In [50]:
df.loc[df.x > 5, ['z', 8]]

,z,8
c,12,13
d,17,18
e,22,23


Keeping all rows and columns of 'z' and '8'?

In [53]:
df.loc[:, ['z', 8]]

,z,8
a,2,3
b,7,8
c,12,13
d,17,18
e,22,23


Keeping the rows that are the top N largest or top n& largest of the x column?

To get the top 3 largest of the 'x' column:

In [54]:
df.nlargest(3,'x')

,x,y,z,8,9
e,20,21,22,23,24
d,15,16,17,18,19
c,10,11,12,13,14


To get the 3 smallest of the 'x' column:

In [55]:
df.nsmallest(3, 'x')

,x,y,z,8,9
a,0,1,2,3,4
b,5,6,7,8,9
c,10,11,12,13,14


To get the top 10% largest of the 'x' column:

In [57]:
df.loc[df.x > df.x.quantile(0.9)]

,x,y,z,8,9
b,5,6,7,8,9
c,10,11,12,13,14
d,15,16,17,18,19
e,20,21,22,23,24


## SettingWithCopyWarning in Pandas

The SettingWithCopyWarning was created to flag potentially confusing "chained" assignments, such as the following, which does not always work as expected, particularly when the first selection returns a copy.

The problem with chained assignment, is that *it is generally difficult to predict whether a view or a copy is returned*, so this largely becomes an issue when you are attempting to assign values back, or Python is confused whether you want to change the value in the copy or the original object.

In [72]:
df = pd.DataFrame(np.arange(25).reshape(5, 5),  
                      index=list('abcde'), 
                      columns=['x','y','z', 8, 9]
                      )
print(df)

    x   y   z   8   9
a   0   1   2   3   4
b   5   6   7   8   9
c  10  11  12  13  14
d  15  16  17  18  19
e  20  21  22  23  24


A task: get a subset of the rows with x column greater than 2, and then set the y column as 'new_val'. Return this subset.

In [73]:
df[df['x'] > 2]['y'] = 'new_val'  # new_val not set in df
print(df)

    x   y   z   8   9
a   0   1   2   3   4
b   5   6   7   8   9
c  10  11  12  13  14
d  15  16  17  18  19
e  20  21  22  23  24


/tmp/ipykernel_9368/1265202580.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df['x'] > 2]['y'] = 'new_val'  # new_val not set in df


The warning offers a suggestion to rewrite as follows. However, this is not what you want.

In [74]:
df.loc[df['x'] > 2,'y'] = 'new_val'  
print(df)

    x        y   z   8   9
a   0        1   2   3   4
b   5  new_val   7   8   9
c  10  new_val  12  13  14
d  15  new_val  17  18  19
e  20  new_val  22  23  24


In [64]:
# This is what we want
df = df[df['x'] > 2]
df.y = 'new_val'
print(df)

    x        y   z   8   9
b   5  new_val   7   8   9
c  10  new_val  12  13  14
d  15  new_val  17  18  19
e  20  new_val  22  23  24


### There are a couple ways of directly silencing this warning:

1. (recommended) Use loc to slice subsets and explicitly create copies

In [70]:
df = pd.DataFrame(np.arange(25).reshape(5, 5),  
                      index=list('abcde'), 
                      columns=['x','y','z', 8, 9]
                      )
print(df)

df2 = df.loc[:, ['x']] # if the result of loc is assigned to another df, then it creates copy
df2['x'] /= 2     # Does not raise
print(df)
print(df2)

    x   y   z   8   9
a   0   1   2   3   4
b   5   6   7   8   9
c  10  11  12  13  14
d  15  16  17  18  19
e  20  21  22  23  24
    x   y   z   8   9
a   0   1   2   3   4
b   5   6   7   8   9
c  10  11  12  13  14
d  15  16  17  18  19
e  20  21  22  23  24
      x
a   0.0
b   2.5
c   5.0
d   7.5
e  10.0


2. Change pd.options.mode.chained_assignment Can be set to None, "warn", or "raise". "warn" is the default. None will suppress the warning entirely, and "raise" will throw a SettingWithCopyError, preventing the operation from going through.

3. Make a deepcopy

In [78]:
df2 = df[['x']].copy(deep=True)
df2['x'] /= 2
print(df2)

      x
a   0.0
b   2.5
c   5.0
d   7.5
e  10.0


## Another question: for df, I want to assign values in col "x" > 5 to 1000. What is the right way?

In [80]:
df.x[df.x > 5] = 1000         # works, because df.A returns a view
df[df.x > 5]['x'] = 1000      # does not work
df.loc[df.x > 5]['x'] = 1000   # does not work
df.loc[df.x > 5, 'x'] = 1000   # correct way of using df. It creates a view, not copy

/tmp/ipykernel_9368/1419058441.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.x[df.x > 5] = 1000         # works, because df.A returns a view
/tmp/ipykernel_9368/1419058441.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df.x > 5]['x'] = 1000      # does not work
/tmp/ipykernel_9368/1419058441.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

## Make sure that you create a copy if you want to drop a column or change values

In [83]:
df = pd.DataFrame(np.arange(25).reshape(5, 5),  
                      index=list('abcde'), 
                      columns=['x','y','z', 8, 9]
                      )
df2 = df.copy() # the behaviour of copy() is not guaranteed (copy or view), depending on the system and object size
df2.loc[df2.x == 5, 'y'] = 123
print(df2)
print(df)
# looks like a copy is made here

    x    y   z   8   9
a   0    1   2   3   4
b   5  123   7   8   9
c  10   11  12  13  14
d  15   16  17  18  19
e  20   21  22  23  24
    x   y   z   8   9
a   0   1   2   3   4
b   5   6   7   8   9
c  10  11  12  13  14
d  15  16  17  18  19
e  20  21  22  23  24


In [84]:
# To explicitly create a copy, using copy(deep = True)
df2 = df.copy(deep=True) # the behaviour of copy() is not guaranteed (copy or view), depending on the system and object size
df2.loc[df2.x == 5, 'y'] = 123
print(df2)
print(df)

    x    y   z   8   9
a   0    1   2   3   4
b   5  123   7   8   9
c  10   11  12  13  14
d  15   16  17  18  19
e  20   21  22  23  24
    x   y   z   8   9
a   0   1   2   3   4
b   5   6   7   8   9
c  10  11  12  13  14
d  15  16  17  18  19
e  20  21  22  23  24


In [88]:
# If you drop a column of a DataFrame, which is a view of another DataFrame, you will get a warning.
df = pd.DataFrame(np.arange(25).reshape(5, 5),  
                      index=list('abcde'), 
                      columns=['x','y','z', 8, 9]
                      )
df2 = df[['x', 'y']] # return a view or copy
df2 = df[df.x>5] # same problem, return a view or copy
df2 = df.loc[df.x > 5,:] # same problem, return a view or copy 
df2.drop('x', axis=1, inplace=True)
print(df)
print(df2)

    x   y   z   8   9
a   0   1   2   3   4
b   5   6   7   8   9
c  10  11  12  13  14
d  15  16  17  18  19
e  20  21  22  23  24
    y   z   8   9
c  11  12  13  14
d  16  17  18  19
e  21  22  23  24


/tmp/ipykernel_9368/787217124.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.drop('x', axis=1, inplace=True)


In [91]:
df = pd.DataFrame(np.arange(25).reshape(5, 5),  
                      index=list('abcde'), 
                      columns=['x','y','z', 8, 9]
                      )
df2 = df.loc[df.x > 5,:].copy()
df2.drop('x', axis=1, inplace=True)
print(df)
print(df2)

    x   y   z   8   9
a   0   1   2   3   4
b   5   6   7   8   9
c  10  11  12  13  14
d  15  16  17  18  19
e  20  21  22  23  24
    y   z   8   9
c  11  12  13  14
d  16  17  18  19
e  21  22  23  24


In [86]:
print(df2)

    y
a   1
b   6
c  11
d  16
e  21


Reference

1. https://stackoverflow.com/a/31593712
1. https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas
1. A good discussion of the warning, with many exmaples: https://stackoverflow.com/a/53954986